In [86]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time as time
import os

from sklearn.preprocessing import PolynomialFeatures, StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, LogisticRegressionCV, LogisticRegression, ElasticNet

from sklearn.neighbors import KNeighborsRegressor

from sklearn.feature_selection import SelectKBest, mutual_info_regression

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, cross_validate, \
GridSearchCV, RandomizedSearchCV, ParameterGrid, KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold
from skopt import BayesSearchCV

from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier, \
VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier, AdaBoostRegressor,AdaBoostClassifier
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, Pool

from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, \
accuracy_score, precision_score, confusion_matrix, mean_squared_error, r2_score, mean_absolute_error

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_objective, plot_histogram, plot_convergence

from sklearn import impute
import ast
import itertools as it
from sklearn.tree import export_graphviz 
from six import StringIO
from IPython.display import Image  
import pydotplus

os.getcwd()
os.chdir('/Users/kevin/Downloads/Northwestern University/Data Science/STAT_303-3/Prediction Problems/Datasets')

## Step 0) Read data

In [2]:
train = pd.read_csv('train_regression.csv')
test = pd.read_csv('test_regression.csv')

## Step 1) Data pre-processing

### <font color = 'red'>Pre-processing training data</font>

In [151]:
# Cleaning the columns


first_ten = train.iloc[:, :10]

# Removing: ['id', 'host_location', 'host_neighbourhood']
cleaned_ten = first_ten.drop(columns=['id', 'host_id', 'host_location', 'host_neighbourhood'])

# Converting: ['host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_since']
cleaned_ten['host_response_rate'] = pd.to_numeric(cleaned_ten['host_response_rate'].str.strip('%')) / 100
cleaned_ten['host_acceptance_rate'] = pd.to_numeric(cleaned_ten['host_acceptance_rate'].str.strip('%')) / 100
cleaned_ten['host_is_superhost'] = cleaned_ten['host_is_superhost'].map({'t': 1, 'f': 0})
cleaned_ten['host_since'] = pd.to_datetime(cleaned_ten['host_since'])
cleaned_ten['days_since_host'] = (pd.datetime.now() - cleaned_ten['host_since']).dt.days
cleaned_ten = cleaned_ten.drop(columns=['host_since'])



second_ten = train.iloc[:, 10:20]

# Converting: ['host_has_profile_pic', 'neighbourhood_cleansed', 'host_identity_verified','latitude', 'longitude', 'property_type', 'room_type']
cleaned_twenty = second_ten
neighbourhood_counts = cleaned_twenty.neighbourhood_cleansed.value_counts()
neighbourhoods_to_replace = neighbourhood_counts[neighbourhood_counts < 107].index.tolist()
cleaned_twenty['neighbourhood_cleansed'] = cleaned_twenty['neighbourhood_cleansed'].replace(neighbourhoods_to_replace, 'Other')
cleaned_twenty['num_verifications'] = cleaned_twenty['host_verifications'].apply(lambda x: len(ast.literal_eval(x)))
cleaned_twenty = cleaned_twenty.drop(columns=['host_verifications'])
cleaned_twenty['host_has_profile_pic'] = cleaned_twenty['host_has_profile_pic'].map({'t': 1, 'f': 0})
cleaned_twenty['host_identity_verified'] = cleaned_twenty['host_identity_verified'].map({'t': 1, 'f': 0})
cleaned_twenty['latitude'] = pd.to_numeric(cleaned_twenty['latitude'])
cleaned_twenty['longitude'] = pd.to_numeric(cleaned_twenty['longitude'])
cleaned_twenty['property_category'] = "Entire property"
cleaned_twenty.loc[cleaned_twenty['property_type'].str.contains('room', case=False), 'property_category'] = 'Room'
cleaned_twenty = cleaned_twenty.drop(columns=['property_type'])



third_ten = train.iloc[:, 20:30]

# Converting: ['bathrooms_text', 'price']

third_ten['bathrooms_text'] = third_ten['bathrooms_text'].replace({"Half-bath": "0.5", "Shared half-bath": "0.5", "Private half-bath": "0.5"})
third_ten['num_bathrooms'] = third_ten['bathrooms_text'].str.extract(r'(\d+(\.\d+)?)')[0].astype(float)
cleaned_third = third_ten.drop(columns=['bathrooms_text'])
cleaned_third['price'] = cleaned_third['price'].str.replace('[$,]', '', regex=True).astype(float)



fourth_ten = train.iloc[:, 30:40]
fourth_ten.dtypes

# Removing: ['first_review']
# Converting: ['has_availability']

cleaned_fourth = fourth_ten.drop(columns=['first_review'])
cleaned_fourth['has_availability'] = cleaned_fourth['has_availability'].map({'t': 1, 'f': 0})



fifth_ten = train.iloc[:, 40:50]
fifth_ten

# Removing: ['last_review']
# Converting: ['instant_bookable']

cleaned_fifth = fifth_ten.drop(columns=['last_review'])
cleaned_fifth['instant_bookable'] = cleaned_fifth['instant_bookable'].map({'t': 1, 'f': 0})



last_four = train.iloc[:, 50:]

In [152]:
# Combining the cleaned datasets

cleaned_train = pd.concat([cleaned_ten, cleaned_twenty, cleaned_third, cleaned_fourth, cleaned_fifth, last_four], axis=1)

In [153]:
columns_with_missing = ['num_bathrooms', 'reviews_per_month', 'host_is_superhost', 
                        'review_scores_rating', 'host_response_rate', 
                        'host_acceptance_rate', 'beds', 'review_scores_communication', 
                        'review_scores_cleanliness', 'review_scores_accuracy', 
                        'review_scores_value', 'review_scores_location', 'review_scores_checkin']

In [154]:
# Computing the missing values of dummy variables using mode

cleaned_train['host_is_superhost'].fillna(cleaned_train['host_is_superhost'].mode()[0], inplace=True)
cleaned_train['host_response_time'].fillna(cleaned_train['host_response_time'].mode()[0], inplace=True)

In [155]:
# Computing the missing values of numeric variables using KNN

knn_imputer = impute.KNNImputer(n_neighbors=10)
cleaned_train_imputed = knn_imputer.fit_transform(cleaned_train[columns_with_missing])
cleaned_train_imputed_df = pd.DataFrame(cleaned_train_imputed, columns=columns_with_missing)
cleaned_train[columns_with_missing] = cleaned_train_imputed_df

In [156]:
to_be_removed = ['review_scores_communication','review_scores_cleanliness', 'number_of_reviews_l30d', \
                                'review_scores_accuracy', 'review_scores_value','review_scores_location', \
                                'review_scores_checkin', 'minimum_minimum_nights', 'maximum_minimum_nights', \
                                'minimum_maximum_nights', 'maximum_maximum_nights', 'availability_60', \
                                'availability_90', 'availability_365','calculated_host_listings_count',
                                'calculated_host_listings_count_entire_homes', 'host_listings_count']

In [157]:
y_train = np.log(cleaned_train.price)
X_train = cleaned_train.drop(columns = 'price')

In [158]:
X_train_onehot = pd.get_dummies(X_train, drop_first = True)

In [159]:
X_train_non_redundant = X_train.copy()
X_train_non_redundant.drop(columns = to_be_removed, inplace = True)
X_train_non_redundant = pd.get_dummies(X_train_non_redundant, drop_first = True)

#### <font color = 'black'>PolynomialFeatures</font>

**Non redundant**

In [160]:
poly = PolynomialFeatures(2, include_bias = False)
X_train_poly = poly.fit_transform(X_train_non_redundant)

In [161]:
X_train_non_scaled_poly_df = pd.DataFrame(X_train_poly, columns = poly.get_feature_names_out(X_train_non_redundant.columns))

**Redundant**

In [162]:
poly_redundant = PolynomialFeatures(2, include_bias = False)
X_train_redundant_poly = poly_redundant.fit_transform(X_train_onehot)
X_train_redundant_poly_df = pd.DataFrame(X_train_redundant_poly, columns = poly_redundant.get_feature_names_out(X_train_onehot.columns))

### <font color = 'red'>Pre-processing test data</font>

In [133]:
# Cleaning the test data

first_ten = test.iloc[:, :10]
first_ten

# Removing: ['host_location', 'host_neighbourhood']
# Converting: ['host_response_rate', 'host_acceptance_rate', 'host_is_superhost']

cleaned_ten = first_ten.drop(columns=['host_location', 'host_neighbourhood'])

cleaned_ten['host_response_rate'] = pd.to_numeric(cleaned_ten['host_response_rate'].str.strip('%')) / 100
cleaned_ten['host_acceptance_rate'] = pd.to_numeric(cleaned_ten['host_acceptance_rate'].str.strip('%')) / 100
cleaned_ten['host_is_superhost'] = cleaned_ten['host_is_superhost'].map({'t': 1, 'f': 0})
cleaned_ten['host_since'] = pd.to_datetime(cleaned_ten['host_since'])
cleaned_ten['days_since_host'] = (pd.datetime.now() - cleaned_ten['host_since']).dt.days
cleaned_ten = cleaned_ten.drop(columns=['host_since'])



second_ten = test.iloc[:, 10:20]

# Consider removing: []
# Consider converting: ['host_has_profile_pic', 'host_identity_verified','latitude', 'longitude', 'property_type', 'room_type']

cleaned_twenty = second_ten
neighbourhood_counts = cleaned_twenty.neighbourhood_cleansed.value_counts()
neighbourhoods_to_replace = neighbourhood_counts[neighbourhood_counts < 64].index.tolist()
cleaned_twenty['neighbourhood_cleansed'] = cleaned_twenty['neighbourhood_cleansed'].replace(neighbourhoods_to_replace, 'Other')
cleaned_twenty['num_verifications'] = cleaned_twenty['host_verifications'].apply(lambda x: len(ast.literal_eval(x)))
cleaned_twenty = cleaned_twenty.drop(columns=['host_verifications'])
cleaned_twenty['host_has_profile_pic'] = cleaned_twenty['host_has_profile_pic'].map({'t': 1, 'f': 0})
cleaned_twenty['host_identity_verified'] = cleaned_twenty['host_identity_verified'].map({'t': 1, 'f': 0})
cleaned_twenty['latitude'] = pd.to_numeric(cleaned_twenty['latitude'])
cleaned_twenty['longitude'] = pd.to_numeric(cleaned_twenty['longitude'])
cleaned_twenty['property_category'] = "Entire property"
cleaned_twenty.loc[cleaned_twenty['property_type'].str.contains('room', case=False), 'property_category'] = 'Room'
cleaned_twenty = cleaned_twenty.drop(columns=['property_type'])



third_ten = test.iloc[:, 20:30]

# Converting: ['bathrooms_text']

third_ten['bathrooms_text'] = third_ten['bathrooms_text'].replace({"Half-bath": "0.5", "Shared half-bath": "0.5", "Private half-bath": "0.5"})
third_ten['num_bathrooms'] = third_ten['bathrooms_text'].str.extract(r'(\d+(\.\d+)?)')[0].astype(float)
cleaned_third = third_ten.drop(columns=['bathrooms_text'])



fourth_ten = test.iloc[:, 30:40]

# Removing: ['first_review', 'last_review']
# Converting: ['has_availability']

cleaned_fourth = fourth_ten.drop(columns=['first_review', 'last_review'])
cleaned_fourth['has_availability'] = cleaned_fourth['has_availability'].map({'t': 1, 'f': 0})



fifth_ten = test.iloc[:, 40:50]

# Consider removing: []
# Consider converting: ['instant_bookable']

fifth_ten['instant_bookable'] = fifth_ten['instant_bookable'].map({'t': 1, 'f': 0})

last_three = test.iloc[:, 50:]

In [134]:
# Combining the test datasets
cleaned_test = pd.concat([cleaned_ten, cleaned_twenty, cleaned_third, cleaned_fourth, fifth_ten, last_three], axis=1)

In [135]:
copy_ct = cleaned_test.copy()

copy_ct['host_is_superhost'].fillna(copy_ct['host_is_superhost'].mode()[0], inplace=True)
copy_ct['host_response_time'].fillna(copy_ct['host_response_time'].mode()[0], inplace=True)

columns_with_missing = ['num_bathrooms', 'reviews_per_month', 'host_is_superhost', 
                        'review_scores_rating', 'host_response_rate', 
                        'host_acceptance_rate', 'beds', 'review_scores_communication', 
                        'review_scores_cleanliness', 'review_scores_accuracy', 
                        'review_scores_value', 'review_scores_location', 'review_scores_checkin']

knn_imputer = impute.KNNImputer(n_neighbors=10)
copy_ct_imputed = knn_imputer.fit_transform(copy_ct[columns_with_missing])
copy_ct_imputed_df = pd.DataFrame(copy_ct_imputed, columns=columns_with_missing)
copy_ct[columns_with_missing] = copy_ct_imputed_df

In [136]:
X_test_non_redundant = copy_ct.drop(columns = to_be_removed)
X_test_non_redundant = pd.get_dummies(X_test_non_redundant, drop_first = True)
X_test_non_redundant = X_test_non_redundant.iloc[:, 2:]

#### <font color = 'black'>PolynomialFeatures</font>

In [137]:
poly_test = PolynomialFeatures(2, include_bias = False)
poly_test.fit(X_test_non_redundant)
X_test_poly = poly_test.transform(X_test_non_redundant)

In [138]:
X_test_non_scaled_poly_df = pd.DataFrame(X_test_poly, columns = poly_test.get_feature_names_out(X_test_non_redundant.columns))

In [139]:
poly_test_all = PolynomialFeatures(2, include_bias = False)
poly_test_all.fit(pd.get_dummies(copy_ct, drop_first = True))
X_test_all = poly_test_all.transform(pd.get_dummies(copy_ct, drop_first = True))
X_test_all_df = pd.DataFrame(X_test_all, columns = poly_test_all.get_feature_names_out(pd.get_dummies(copy_ct, drop_first = True).columns))

## 2) Hyperparameter tuning

### How many attempts did it take you to tune the model hyperparameters?

### Which tuning method did you use (grid search / Bayes search / etc.)?

### What challenges did you face while tuning the hyperparameters, and what actions did you take to address those challenges?

### How many hours did you spend on hyperparameter tuning?

### Hyperparameter tuning code

## <font color = 'red'>Variable selection</font>

### Lasso

In [21]:
scaler = StandardScaler()
scaler.fit(X_train_poly)
X_train_scaled = scaler.transform(X_train_poly)

In [22]:
start_time = time.time()

# Lasso for variable selection
alphas = np.logspace(-1,-5,200)
lassocv = LassoCV(alphas = alphas, cv = 10, max_iter = 1000)
lassocv.fit(X_train_scaled, y_train)

print(lassocv.alpha_)

print("Time taken = ", np.round((time.time() - start_time)/60,2), "minutes")

0.002833096101839324
Time taken =  7.12 minutes


In [23]:
lasso = Lasso(alpha = lassocv.alpha_)
lasso.fit(X_train_scaled, y_train)
coefficients = {}
for i in range(len(lasso.coef_)):
    coefficients[poly.get_feature_names_out()[i]] = lasso.coef_[i]

In [24]:
coefficients = pd.Series(data = coefficients)
non_zero_coefficients = coefficients[coefficients != 0]

In [54]:
non_zero_coefficients

latitude                                                        0.052021
longitude                                                       0.018012
accommodates                                                    0.090814
beds                                                            0.026985
num_bathrooms                                                   0.176340
                                                                  ...   
neighbourhood_cleansed_Near West Side room_type_Private room   -0.013930
neighbourhood_cleansed_Uptown property_category_Room            0.013885
neighbourhood_cleansed_West Town room_type_Hotel room           0.031783
neighbourhood_cleansed_West Town room_type_Shared room          0.003070
neighbourhood_cleansed_West Town property_category_Room         0.023231
Length: 283, dtype: float64

### SelectKBest

In [69]:
selector = SelectKBest(score_func=mutual_info_regression, k=300)
X_train_selectkbest_df = pd.DataFrame(selector.fit_transform(X_train_redundant_poly_df, y_train), columns = selector.get_feature_names_out())

In [59]:
X_train_selectkbest_df

,host_acceptance_rate,days_since_host,host_total_listings_count,latitude,longitude,accommodates,beds,minimum_nights_avg_ntm,num_bathrooms,review_scores_rating,...,reviews_per_month^2,reviews_per_month host_response_time_within an hour,reviews_per_month room_type_Private room,reviews_per_month property_category_Room,host_response_time_within an hour room_type_Private room,host_response_time_within an hour property_category_Room,neighbourhood_cleansed_Other property_category_Room,room_type_Private room^2,room_type_Private room property_category_Room,property_category_Room^2
0,0.95,2241.0,13.0,41.794240,-87.661380,1.0,1.0,32.0,1.0,4.94,...,0.108900,0.330,0.330,0.330,1.0,1.0,1.0,1.0,1.0,1.0
1,0.97,3444.0,2346.0,41.892749,-87.620711,12.0,3.0,32.0,3.0,4.74,...,1.643524,1.282,1.282,1.282,1.0,1.0,0.0,1.0,1.0,1.0
2,1.00,1542.0,1.0,41.948160,-87.652309,6.0,3.0,2.0,1.0,4.87,...,7.728400,2.780,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.98,3141.0,58.0,41.801030,-87.597160,2.0,1.0,2.0,1.0,4.08,...,13.032100,3.610,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,1.00,1150.0,75.0,41.855770,-87.624130,6.0,3.0,2.0,2.0,4.80,...,10.497600,3.240,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.84,1969.0,2.0,42.012880,-87.672190,2.0,1.0,4.0,1.0,4.98,...,1.742400,0.000,1.320,1.320,0.0,0.0,1.0,1.0,1.0,1.0
4996,0.89,537.0,9.0,41.946662,-87.648015,2.0,1.0,2.0,1.0,4.71,...,171.872100,13.110,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4997,0.97,2499.0,4.0,41.987076,-87.717232,4.0,2.0,3.0,1.0,4.94,...,0.490000,0.700,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4998,0.97,2720.0,6438.0,41.937821,-87.650831,4.0,2.0,350.3,1.0,4.73,...,2.064969,1.437,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0


In [131]:
#K-fold cross validation to find optimal parameters for CatBoost regressor
start_time = time.time()
param_grid = {'max_depth': [6,8,10, 12],
              'num_leaves': [31, 37, 42, 46],
              'learning_rate': [0.01, 0.02, 0.03, 0.05],
               'reg_lambda':[2,3,5,7],
                'n_estimators':[1500, 1750, 2000, 2250],
                'subsample': [0.45, 0.5, 0.55],
             'colsample_bylevel': [0.25, 0.3, 0.36, 0.43, 0.5]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
optimal_params = RandomizedSearchCV(estimator=CatBoostRegressor(random_state=1, verbose=False, 
                            grow_policy='Lossguide'),                                                       
                             param_distributions = param_grid, n_iter = 200,
                             verbose = 1,random_state = 1, scoring='neg_root_mean_squared_error',
                             n_jobs=-1,
                             cv = cv)
optimal_params.fit(X_train_selectkbest_df,y_train)
print("Optimal parameter values =", optimal_params.best_params_)
print("Optimal cross validation RMSE = ",optimal_params.best_score_)
print("Time taken = ", round((time.time()-start_time)/60), " minutes")

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.war

Optimal parameter values = {'subsample': 0.55, 'reg_lambda': 5, 'num_leaves': 46, 'n_estimators': 1500, 'max_depth': 8, 'learning_rate': 0.03, 'colsample_bylevel': 0.3}
Optimal cross validation RMSE =  -0.3637580570124931
Time taken =  470  minutes


In [132]:
catboost_selectkbest_model = CatBoostRegressor(grow_policy='Lossguide', random_state=1, verbose=0, subsample=0.55, reg_lambda=5, num_leaves=46,
                                  n_estimators=1500, max_depth=8, learning_rate=0.03, colsample_bylevel=0.3).fit(X_train_selectkbest_df,y_train)

### CatBoost

**First, tune a CatBoost model**

In [87]:
catboost_model = CatBoostRegressor(grow_policy='Lossguide', random_state=1, verbose=0, subsample=0.45, reg_lambda=3, num_leaves=42,
                                  n_estimators=2250, max_depth=10, learning_rate=0.02, colsample_bylevel=0.38).fit(X_train_redundant_poly_df,y_train)

In [148]:
# Assume X_train_redundant_poly_df and y_train are already defined

# Define the CatBoost model
catboost_model = CatBoostRegressor(
    grow_policy='Lossguide', random_state=1, verbose=0, subsample=0.45, 
    reg_lambda=3, num_leaves=42, n_estimators=2250, max_depth=10, 
    learning_rate=0.02, colsample_bylevel=0.38
)

# Range of number of features to select
num_features_to_select_range = list(range(150, 501, 50))

# Variables to store the best number of features and the corresponding performance
best_num_features = None
best_rmse = float('inf')
best_features = None

# Perform feature selection and model training for each value in the range
start_time = time.time()

for num_features in num_features_to_select_range:
    
    # Perform feature selection
    selected_features = catboost_model.select_features(
        X_train_redundant_poly_df, y=y_train,
        algorithm='RecursiveByLossFunctionChange',
        features_for_select=list(X_train_redundant_poly_df.columns),
        num_features_to_select=num_features,
        steps=1,
        train_final_model=False,
        logging_level='Silent'
    )
    
    # Get the selected feature names
    selected_feature_names = selected_features['selected_features_names']
    
    # Train-test split
    X_train_selected = X_train_redundant_poly_df[selected_feature_names]
    X_train, X_test, y_train, y_test = train_test_split(X_train_selected, y_train, test_size=0.25, random_state=1)
    
    # Train the model with the selected features
    train_pool = Pool(X_train, y_train)
    test_pool = Pool(X_test, y_test)
    catboost_model.fit(train_pool, eval_set=test_pool, use_best_model=True, early_stopping_rounds=10)
    
    # Predict and calculate RMSE
    y_pred = catboost_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print(f"Num features: {num_features}, RMSE: {rmse}, Time taken: {np.round((time.time() - start_time) / 60, 2)} minutes")
    
    # Update the best number of features and RMSE if current RMSE is better
    if rmse < best_rmse:
        best_rmse = rmse
        best_num_features = num_features
        best_features = selected_feature_names

print(f"Best number of features: {best_num_features}, Best RMSE: {best_rmse}")
print("Time taken = ", np.round((time.time() - start_time)/60,2), "minutes")

Num features: 150, RMSE: 0.35409223867115874, Time taken: 150.79 minutes


CatBoostError: Length of label=3750 and length of data=5000 is different.

In [163]:
start_time = time.time()

# Perform feature selection
selected_features = catboost_model.select_features(
    X_train_redundant_poly_df, y = y_train,
    algorithm='RecursiveByLossFunctionChange',
    features_for_select=list(X_train_redundant_poly_df.columns),
    num_features_to_select=150,  # Number of features to select
    steps=1,  # Number of features to remove at each step
    train_final_model=False,
    logging_level='Silent'
)

print("Time taken = ", np.round((time.time() - start_time)/60,2), "minutes")

Time taken =  74.4 minutes


In [164]:
# Get the selected features
selected_feature_indices = selected_features['selected_features_names']
print("Selected features:", selected_feature_indices)

# Create a new DataFrame with the selected features
X_train_catboost_selected = X_train_redundant_poly_df[selected_feature_indices]

Selected features: ['latitude', 'longitude', 'accommodates', 'host_response_rate host_total_listings_count', 'host_response_rate latitude', 'host_response_rate longitude', 'host_response_rate accommodates', 'host_response_rate reviews_per_month', 'host_acceptance_rate accommodates', 'host_acceptance_rate beds', 'host_acceptance_rate minimum_nights', 'host_acceptance_rate num_bathrooms', 'host_acceptance_rate availability_30', 'host_acceptance_rate availability_90', 'host_acceptance_rate calculated_host_listings_count', 'host_acceptance_rate reviews_per_month', 'host_acceptance_rate neighbourhood_cleansed_Other', 'host_is_superhost availability_365', 'host_is_superhost review_scores_accuracy', 'host_is_superhost review_scores_cleanliness', 'host_listings_count days_since_host', 'host_listings_count minimum_nights', 'host_listings_count minimum_minimum_nights', 'host_listings_count maximum_minimum_nights', 'host_listings_count maximum_maximum_nights', 'host_listings_count minimum_nights_

In [165]:
X_train_catboost_selected

,latitude,longitude,accommodates,host_response_rate host_total_listings_count,host_response_rate latitude,host_response_rate longitude,host_response_rate accommodates,host_response_rate reviews_per_month,host_acceptance_rate accommodates,host_acceptance_rate beds,...,number_of_reviews_ltm calculated_host_listings_count,number_of_reviews_l30d review_scores_communication,review_scores_accuracy neighbourhood_cleansed_Other,review_scores_cleanliness review_scores_location,review_scores_checkin calculated_host_listings_count,review_scores_checkin neighbourhood_cleansed_Lake View,calculated_host_listings_count reviews_per_month,calculated_host_listings_count neighbourhood_cleansed_Other,reviews_per_month neighbourhood_cleansed_Near West Side,reviews_per_month neighbourhood_cleansed_Other
0,41.794240,-87.661380,1.0,12.48,40.122470,-84.154925,0.96,0.3168,0.95,0.95,...,0.0,0.00,5.00,20.929200,45.000,0.000,2.970,9.0,0.0,0.33
1,41.892749,-87.620711,12.0,2346.00,41.892749,-87.620711,12.00,1.2820,11.64,2.91,...,0.0,0.00,0.00,22.349166,280.546,0.000,74.356,0.0,0.0,0.00
2,41.948160,-87.652309,6.0,1.00,41.948160,-87.652309,6.00,2.7800,6.00,3.00,...,14.0,5.00,0.00,24.650000,5.000,5.000,2.780,0.0,0.0,0.00
3,41.801030,-87.597160,2.0,58.00,41.801030,-87.597160,2.00,3.6100,1.96,0.98,...,715.0,12.69,4.08,18.160000,240.900,0.000,198.550,55.0,0.0,3.61
4,41.855770,-87.624130,6.0,75.00,41.855770,-87.624130,6.00,3.2400,6.00,3.00,...,3108.0,14.55,0.00,22.515000,357.420,0.000,239.760,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,42.012880,-87.672190,2.0,1.76,36.971334,-77.151527,1.76,1.1616,1.68,0.84,...,22.0,0.00,4.98,24.850200,10.000,0.000,2.640,2.0,0.0,1.32
4996,41.946662,-87.648015,2.0,9.00,41.946662,-87.648015,2.00,13.1100,1.78,0.89,...,472.0,88.20,0.00,23.953800,38.000,4.750,104.880,0.0,0.0,0.00
4997,41.987076,-87.717232,4.0,4.00,41.987076,-87.717232,4.00,0.7000,3.88,1.94,...,30.0,10.00,4.97,24.700900,14.820,0.000,2.100,3.0,0.0,0.70
4998,41.937821,-87.650831,4.0,6438.00,41.937821,-87.650831,4.00,1.4370,3.88,1.94,...,0.0,0.00,0.00,22.634595,3112.763,4.841,923.991,0.0,0.0,0.00


In [117]:
def build_model(df, target):
    """
    Given the dataframe and target, build and return the model
    """
    # Set up target
    y = target
    # Set up train-test split
    X = df
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=1)
    # Identify the categorical features
    #categorical_features = X_train.select_dtypes(exclude=[np.number])
    categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()
    # Create training and test pools for catboost
    train_pool = Pool(X_train, y_train, categorical_features)
    test_pool = Pool(X_test, y_test, categorical_features)
    # Fit the model and calculate RMSE
    model = CatBoostRegressor(grow_policy='Lossguide', random_state=1, verbose=0, subsample=0.45, reg_lambda=3, num_leaves=42,
                                  n_estimators=2250, max_depth=10, learning_rate=0.02, colsample_bylevel=0.38)
    model.fit(X_train, y_train, eval_set=test_pool, cat_features=categorical_features, use_best_model=True, early_stopping_rounds=10)
    rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
    return rmse, model, X_test

In [118]:
def get_dropped_feature(model, X_test):
    explainer = shap.Explainer(model)
    shap_values = explainer(X_test)
    feature_importance = shap_values.abs.mean(0).values
    importance_df = pd.DataFrame({'features': X_test.columns,
                                  'importance': feature_importance})
    importance_df.sort_values(by='importance', ascending=False, inplace=True)
    return importance_df['features'].iloc[-1]   

In [119]:
def backward_selection(df, target, max_features=None):
    """
    This function uses the SHAP importance from a catboost model
    to incrementally remove features from the training set until the RMSE no longer improves.
    This function returns the dataframe with the features that give the best RMSE.
    Return at most max_features.
    """
    # get baseline RMSE
    select_df = df.copy()
    total_features = df.shape[1]
    rmse, model, X_test = build_model(select_df, target)
    print(f"{rmse} with {select_df.shape[1]}")
    last_rmse = rmse
    
    # Drop least important feature and recalculate model peformance
    if max_features is None:
        max_features = total_features-1
        
    for num_features in range(total_features-1, 1, -1):
        # Trim features
        dropped_feature = get_dropped_feature(model, X_test)
        tmp_df = select_df.drop(columns=[dropped_feature])

        # Rerun modeling
        rmse, model, X_test = build_model(tmp_df, target)
        print(f"{rmse} with {tmp_df.shape[1]}")
        if (num_features < max_features) and (rmse > last_rmse):
            # RMSE increased, return last dataframe
            return select_df
        else:
            # RMSE improved, continue dropping features
            last_rmse = rmse
            select_df = tmp_df
    return select_df

In [120]:
start_time = time.time()
reduced_df = backward_selection(X_train_redundant_poly_df, y_train, max_features=350)
reduced_df.shape[1]
print("Time taken = ", np.round((time.time() - start_time)/60,2), "minutes")

0.3583729875107813 with 2144


NameError: name 'shap' is not defined

## Step 3) Developing the model

**Test: Using an untuned CatBoost on SelectKBest dataset**

In [171]:
catboost_without_tuning = CatBoostRegressor(random_state=1, verbose = False).fit(X_train_catboost_selected, y_train)

In [172]:
#untuned_pred = np.exp(catboost_without_tuning.predict(X_test_all_df.loc[:, X_train_selectkbest_df.columns]))
untuned_pred = np.exp(catboost_without_tuning.predict(X_test_all_df.loc[:, list(X_train_catboost_selected.columns)]))

In [173]:
untuned_pred

array([172.57430386, 134.5209258 , 359.66740578, ...,  84.33943559,
        93.38951722, 163.25807633])

In [143]:
tuned_pred = np.exp(catboost_selectkbest_model.predict(X_test_all_df.loc[:, X_train_selectkbest_df.columns]))

### <font color = 'red'>Ensembling CatBoost and GradientBoost</font>

In [25]:
combined_df = pd.concat([X_train_onehot, X_train_non_scaled_poly_df.loc[:, non_zero_coefficients.index]], axis=1)
combined_train_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

In [26]:
col_set1 = X_train_onehot.columns
col_set2 = X_train_non_scaled_poly_df.loc[:, non_zero_coefficients.index].columns

In [27]:
cat = CatBoostRegressor(random_state=1, verbose=False)
gb = GradientBoostingRegressor(random_state=1, loss = 'huber')

In [28]:
cat_pipe1 = Pipeline([
    ('column_transformer', ColumnTransformer([('cat1_transform', 'passthrough', col_set1)], remainder='drop')),
    ('cat1', cat)
])

cat_pipe2 = Pipeline([
    ('column_transformer', ColumnTransformer([('cat2_transform', 'passthrough', col_set2)], remainder='drop')),
    ('cat2', cat)
])

gb_pipe1 = Pipeline([
    ('column_transformer', ColumnTransformer([('gb1_transform', 'passthrough', col_set1)], remainder='drop')),
    ('gb1', gb)
])

gb_pipe2 = Pipeline([
    ('column_transformer', ColumnTransformer([('gb2_transform', 'passthrough', col_set2)], remainder='drop')),
    ('gb2', gb)
])

In [29]:
start_time = time.time()

en_pipeline = StackingRegressor(estimators = [('cat1', cat_pipe1),('cat2', cat_pipe2),
                                        ('gb1', gb_pipe1), ('gb2', gb_pipe2)],
                     final_estimator=LinearRegression(), cv = KFold(n_splits = 10, shuffle = True, random_state=1))

en_pipeline.fit(combined_train_df, y_train)

print("Time taken = ", np.round((time.time() - start_time)/60,2), "minutes")

Time taken =  3.05 minutes


### Optimal hyperparameter values

**I did not tune the hyperparameters of the CatBoost model or the GradientBoost model, meaning that both models simply used their default values. However, I did use the Huber loss function for the GradientBoost model. When I ensembled the models using `StackingRegressor`, I tuned the weights given to each of the models when making the final prediction; the optimal weights are as follows: `cat_pipe1`: 0.60236223, `cat_pipe2`: 0.53961064, `gb_pipe1`: -0.10894311, `gb_pipe2`: -0.02674319**

In [30]:
en_pipeline.final_estimator_.coef_

array([ 0.60236223,  0.53961064, -0.10894311, -0.02674319])

## Step 4) Ad-hoc steps for further improving model accuracy

In [31]:
pipeline_pred = np.exp((en_pipeline.predict(X_test_all_df.loc[:, combined_train_df.columns])))

In [39]:
pipeline_pred (~108/109)

array([178.86310864, 141.48124472, 358.20633849, ...,  79.17111358,
        88.57938117, 134.43386379])

In [145]:
untuned_pred

array([139.66575198, 139.57035912, 345.5667462 , ...,  72.45813521,
       102.59303862, 144.76731076])

In [144]:
tuned_pred

array([211.80320721, 134.34651985, 326.75807622, ...,  68.32267018,
       112.00302502, 145.81607724])

In [146]:
(tuned_pred + untuned_pred) / 2

array([175.73447959, 136.95843949, 336.16241121, ...,  70.3904027 ,
       107.29803182, 145.291694  ])

In [174]:
untuned_pred

array([172.57430386, 134.5209258 , 359.66740578, ...,  84.33943559,
        93.38951722, 163.25807633])

## Step 5) Exporting the predictions in the format required to submit on Kaggle

In [175]:
copy_ct.insert(1, "predicted", untuned_pred)
to_submit = copy_ct.iloc[:, :2]

In [176]:
to_submit.to_csv('Ensembling - Trial 7 (Untuned CatBoost on CatBoost selected features).csv', index=False)  